In [3]:
import os 
import sys
while os.path.split(os.getcwd())[1] != 'RecSysChallenge2023-Team':
    os.chdir('..')
sys.path.insert(1, os.getcwd())

################################# IMPORT HyperTuning  #################################
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from skopt.space import Real, Integer, Categorical

################################# IMPORT RECOMMENDERS #################################
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.IR_feature_weighting import okapi_BM_25

################################## IMPORT LIBRARIES ##################################
import scipy.sparse as sps
from tqdm import tqdm
import pandas as pd
import numpy as np
import similaripy
import math 

import Daniele.Utils.MatrixManipulation as mm
import MyDataManager as dm

from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout
from sklearn.preprocessing import normalize

URMv = dm.getURMviews()
URMo = dm.getURMopen()

urm = mm.defaultExplicitURM(urmv=URMv,urmo=URMo, normalize=False, add_aug=False)
urm.data = np.ones(len(urm.data))

URMv_train, URMv_test = split_train_in_two_percentage_global_sample(urm, train_percentage = 0.80)

urma = mm.augmentURM(URMv_train)
urma.data = np.ones(len(urma.data))
URMfat = sps.vstack([URMv_train,urma])    


recommender = RP3betaRecommender(URMfat)
#{'topK': 106, 'alpha': 0.6155817289031643, 'beta': 0.33427474623540737, 'normalize_similarity': True} -> MAP 0.0276355
#{'topK': 80, 'alpha': 0.7324323855707952, 'beta': 0.23399909448262007, 'normalize_similarity': True} -> MAP: 0.0281745
recommender.fit(topK= 80, alpha= 0.7324323855707952, beta= 0.23399909448262007, normalize_similarity= True)


100%|██████████| 663370/663370 [00:00<00:00, 1616965.41it/s]


RP3betaRecommender: URM Detected 3461 (12.4%) items with no interactions.
RP3betaRecommender: Similarity column 27968 (100.0%), 4241.63 column/sec. Elapsed time 6.59 sec


In [4]:
evaluator_test = EvaluatorHoldout(URMv_test, [10])
ris, _ = evaluator_test.evaluateRecommender(recommender)
ris

EvaluatorHoldout: Ignoring 319 ( 0.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 41310 (100.0%) in 16.41 sec. Users per second: 2518


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.064895,0.088524,0.076965,0.032796,0.043837,0.200957,0.091304,0.070416,0.395255,0.254216,...,0.992337,0.392227,0.992337,0.057591,9.528339,0.994991,0.133823,0.709683,3.494658,0.297091


In [2]:
evaluator_test = EvaluatorHoldout(URMv_test, [10])
ris, _ = evaluator_test.evaluateRecommender(recommender)
ris

EvaluatorHoldout: Ignoring 349 ( 0.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 41280 (100.0%) in 13.51 sec. Users per second: 3057


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.065879,0.089008,0.077088,0.033266,0.044223,0.202983,0.09215,0.071044,0.398571,0.257036,...,0.991616,0.395229,0.991616,0.059926,9.626564,0.995288,0.139108,0.716957,3.391779,0.299112
